# Tutorial 5: Editing a SIR 3S model safely and effectively

This Tutorial demonstrates how to change SIR 3S models properly by grouping changes and saving them.

The database used for this Tutorial is based on [Network1](https://3sconsult.github.io/sir3stoolkit/examples.html#network-1).

## SIR 3S Installation

In [74]:
SIR3S_SIRGRAF_DIR = r"C:\3S\SIR 3S Entwicklung\SirGraf-90-15-00-16_Quebec_x64" #change to local path

## Imports

Note: The SIR 3S Toolkit requires the Sir3S_Toolkit.dll included in SIR 3S installations (version Quebec and higher).

In [75]:
import sir3stoolkit

The core of sir3stoolkit is a Python wrapper around basic functionality of SIR 3S, offering a low-level access to the creation, modification and simulation of SIR 3S models. In the future pure python subpackages may be added.

In [76]:
from sir3stoolkit.core import wrapper

In [77]:
sir3stoolkit

<module 'sir3stoolkit' from 'C:\\Users\\aUsername\\3S\\sir3stoolkit\\src\\sir3stoolkit\\__init__.py'>

The [wrapper package](https://3sconsult.github.io/sir3stoolkit/references/sir3stoolkit.core.html#sir3stoolkit.core.wrapper.Initialize_Toolkit) has to be has to be initialized with reference to a SIR 3S (SirGraf) installation.

In [78]:
wrapper.Initialize_Toolkit(SIR3S_SIRGRAF_DIR)

## Initialization

The SIR 3S Toolkit contains two classes: [SIR3S_Model](https://3sconsult.github.io/sir3stoolkit/references/sir3stoolkit.core.html#sir3stoolkit.core.wrapper.SIR3S_Model) (model and data) and [SIR3S_View](https://3sconsult.github.io/sir3stoolkit/references/sir3stoolkit.core.html#sir3stoolkit.core.wrapper.SIR3S_View) (depiction in SIR Graf). All SIR 3S functionality is accessed via the methods of these classes.

In [79]:
s3s = wrapper.SIR3S_Model()

Initialization complete


In [80]:
s3s_view = wrapper.SIR3S_View()

Initialization complete


## Open Model

In [81]:
dbFilePath=r"Toolkit_Tutorial5_Model.db3"

In [82]:
s3s.OpenModel(dbName=dbFilePath, 
              providerType=s3s.ProviderTypes.SQLite, 
              Mid="M-1-0-1", 
              saveCurrentlyOpenModel=False, 
              namedInstance="", 
              userID="", 
              password="")

Model is open for further operation


Now the model has been opened. All SIR 3S Toolkit operations now apply to this model until another one is opened.

In [83]:
print(s3s.GetNetworkType()) # check that the correct model is responsive, model we are trying to open was created with type Water

NetworkType.Water


## SIR 3S object types

As we did with non-result values, we need to obtain the internal SIR 3S object types to later pass to our function for obtaining the attribute names.

In [84]:
node_type=s3s.ObjectTypes.Node

In [85]:
pipe_type=s3s.ObjectTypes.Pipe

## GetPropertiesofElementType()

Now, we obtain the internal SIR 3S attribute names. These will be necessary for our value query.

We use [GetPropertiesofElementType()](https://3sconsult.github.io/sir3stoolkit/references/sir3stoolkit.core.html#sir3stoolkit.core.wrapper.SIR3S_Model.GetPropertiesofElementType) to create a list of all properties of nodes and pipes.

In [86]:
node_properties=s3s.GetPropertiesofElementType(node_type)

In [87]:
print(node_properties)

['Name', 'Ktyp', 'Zkor', 'QmEin', 'Lfakt', 'Fkpzon', 'Fkfstf', 'Fkutmp', 'Fkfqps', 'Fkcont', 'Fk2lknot', 'Beschreibung', 'Idreferenz', 'Iplanung', 'Kvr', 'Qakt', 'Xkor', 'Ykor', 'NodeNamePosition', 'ShowNodeName', 'KvrKlartext', 'NumberOfVERB', 'HasBlockConnection', 'Tk', 'Pk', 'IsMarked', 'InVariant', 'GeometriesDiffer', 'SymbolFactor', 'bz.Drakonz', 'bz.Fk', 'bz.Fkpvar', 'bz.Fkqvar', 'bz.Fklfkt', 'bz.PhEin', 'bz.Tm', 'bz.Te', 'bz.PhMin']


## Start/EndTransaction()

The [StartTransaction()](https://3sconsult.github.io/sir3stoolkit/references/sir3stoolkit.core.html#sir3stoolkit.core.wrapper.SIR3S_Model.StartTransaction) function opens a transaction. All changes made to the model afterwards are grouped into this transaction until [EndTransaction()](https://3sconsult.github.io/sir3stoolkit/references/sir3stoolkit.core.html#sir3stoolkit.core.wrapper.SIR3S_Model.EndTransaction) is called. This allows the User to Undo/Redo changes on main UI. A meaningful name should be assigned to each transaction.

In [88]:
nodes=s3s.GetTksofElementType(node_type)

Our goal is to reposition node 0 and 5 in our list. Their current position can be ckecked as follows.

In [89]:
print(f"Tk: {s3s.GetValue(nodes[0], 'Tk')[0]}, X: {s3s.GetValue(nodes[0], 'Xkor')[0]}, Y: {s3s.GetValue(nodes[0], 'Ykor')[0]}")

Tk: 5176769967470325679, X: 200, Y: 700


In [90]:
print(f"Tk: {s3s.GetValue(nodes[5], 'Tk')[0]}, X: {s3s.GetValue(nodes[5], 'Xkor')[0]}, Y: {s3s.GetValue(nodes[5], 'Ykor')[0]}")

Tk: 4906284669343590719, X: 500, Y: 400


Start Transaction

In [91]:
s3s.StartTransaction("Change coords of nodes 5452027739517825954 and 5500965656985444769")

Now you can make changes to the model


In [92]:
s3s.SetValue(nodes[0], "Ykor", str(int(s3s.GetValue(nodes[0], "Ykor")[0])+100)) # add 100 to y coord

Value is set


In [93]:
s3s.SetValue(nodes[5], "Xkor", str(int(s3s.GetValue(nodes[5], "Xkor")[0])-75)) # sub 75 from x coord

Value is set


In [94]:
s3s.SetValue(nodes[5], "Ykor", "450")  # set y coord to 450

Value is set


In [95]:
s3s.EndTransaction()

Nothing done, because no Transaction is being in Use


As of now there appears to be a bug present that Transactions are closed without using EndTransaction().
https://github.com/3SConsult/sir3stoolkit/issues/2

Now we can check our nodes and review the changes made.

In [96]:
print(f"Tk: {s3s.GetValue(nodes[0], 'Tk')[0]}, X: {s3s.GetValue(nodes[0], 'Xkor')[0]}, Y: {s3s.GetValue(nodes[0], 'Ykor')[0]}")

Tk: 5176769967470325679, X: 200, Y: 800


In [97]:
print(f"Tk: {s3s.GetValue(nodes[5], 'Tk')[0]}, X: {s3s.GetValue(nodes[5], 'Xkor')[0]}, Y: {s3s.GetValue(nodes[5], 'Ykor')[0]}")

Tk: 4906284669343590719, X: 425, Y: 450


## Start/EndEditSession()

The [StartEditSession()](https://3sconsult.github.io/sir3stoolkit/references/sir3stoolkit.core.html#sir3stoolkit.core.wrapper.SIR3S_Model.StartEditSession) and [EndEditSession()](https://3sconsult.github.io/sir3stoolkit/references/sir3stoolkit.core.html#sir3stoolkit.core.wrapper.SIR3S_Model.EndEditSession) function are applied the same way as [StartTransaction()](https://3sconsult.github.io/sir3stoolkit/references/sir3stoolkit.core.html#sir3stoolkit.core.wrapper.SIR3S_Model.StartTransaction) and [EndTransaction()](https://3sconsult.github.io/sir3stoolkit/references/sir3stoolkit.core.html#sir3stoolkit.core.wrapper.SIR3S_Model.EndTransaction). The difference is that a session compared to a transaction is designed for bulk changes like changing the values of 40 thousands nodes in a single task.

As so many nodes are a bit overkill for a simple tutorial, we will just show the usage by changing the Zkor of all 11 nodes in our model.

In [98]:
s3s.StartEditSession("Change Zkor of all nodes to 100")

Now you can make changes to the model


In [99]:
for node in nodes:
    s3s.SetValue(node, "Zkor", "100")

Value is set
Value is set
Value is set
Value is set
Value is set
Value is set
Value is set
Value is set
Value is set
Value is set
Value is set


In [100]:
s3s.EndEditSession()

Edit Session has ended. Please open up a new session if you want to make further changes


## SaveChanges()

To save the changes you have made to a model whether grouped into transactions/edit-sessions or not grouped, you can use the [SaveChanges()](https://3sconsult.github.io/sir3stoolkit/references/sir3stoolkit.core.html#sir3stoolkit.core.wrapper.SIR3S_Model.SaveChanges) function to do so.

The below function will therefore save the changes made to the model if uncommented.

In [101]:
#s3s.SaveChanges()

## RefreshViews()

The [RefreshViews()](https://3sconsult.github.io/sir3stoolkit/references/sir3stoolkit.core.html#sir3stoolkit.core.wrapper.SIR3S_Model.RefreshViews) function can be helpful when working with SIR 3S Toolkit in the built-in python scripting environment in SIR Graf. The function will force the presenter (SIR Graf) to refresh it views. If you made changes such as inserting a new object or change visual aspects this function ensures that they will be displayed.

In [102]:
s3s.RefreshViews()

Refresh successful


## General approach to changing models

Below a recommendation on how to structure and encapsulate code that modifies models is given.

In [103]:
s3s.StartTransaction("Transaction Name")  # or EditSession, depending on the 'size' of the change

Now you can make changes to the model


In [104]:
# Make all kinds of changes

In [105]:
s3s.EndTransaction()

Transaction has ended. Please open up a new transaction if you want to make further changes


In [106]:
#s3s.SaveChanges()

In [107]:
s3s.RefreshViews()

Refresh successful


__Next:__ ...